## Reproducibility
1. Clone the repository
2. run fetch_SanchezSaez2020
3. run create_datasets
4. Cringe: I can't use seed in ppca bc my pull request hasn't been accepted. Should I just copy the code?
5. Every section has its notebook, each notebook many tests
6. Each test has a section that builds the data/ part, mainly data preprocessing, big so I cannot put it on github
7. Then once the data is created it's analyzed

### Modules

In [2]:
import sys
import os
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
import pickle as pkl
import importlib
from ppca import PPCA

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_recall_fscore_support, roc_auc_score
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import RFECV

from utils.testers import ss, assess
from utils.reporters import basic_scores, bs_rankings, bs_curves
from utils.helpers import mask50
importlib.reload(utils.reporters)
importlib.reload(utils.testers)
importlib.reload(utils.helpers)

### Data
The training data were lightcurves from ZTF alerts, labeled according to different catalogs as described in the articles.

The publicly available data related to the LC classifier is composed of three .csv files: 
- `labeled_set_lc_classifier_SanchezSaez_2020.csv` (~10MB) aka `labels_file`: the training set with columns ZTF id, label, celestial coordinates, catalog (e.g. MILLIQUAS) and catalog id.
- `features_for_lc_classifier_20200609.csv` (~2.2GB) aka `features_file`: the complete dataset containing the labeled events as well as unlabeled events for testing, with columns ZTF id and the 183 features.
- `ALeRCE_lc_classifier_outputs_ZTF_unlabeled_set_20200609.csv` (~300MB) aka `test_file`: the test set with columns ZTF id, celestial coordinates, probabilities of belonging to each class, predicted class and predicted class probability.

ZTF id is called OID in the files.

In [ ]:
# Convert .csv to .pkl for faster pandas reading, just one time

raw_data_path = "data/raw/"
features_filename = "features_for_lc_classifier_20200609.csv"
labels_filename = "labeled_set_lc_classifier_SanchezSaez_2020.csv"
test_filename = "ALeRCE_lc_classifier_outputs_ZTF_unlabeled_set_20200609.csv"

features_file = pd.read_csv(raw_data_path + features_filename,index_col="oid")
labels_file = pd.read_csv(raw_data_path + labels_filename,index_col="oid")
test_file = pd.read_csv(raw_data_path + test_filename,index_col="oid")

features_file.to_pickle(raw_data_path + "features_file.pkl")
labels_file.to_pickle(raw_data_path + "labels_file.pkl")
test_file.to_pickle(raw_data_path + "test_file.pkl")

In [4]:
# Fast data import

features_file = pd.read_pickle(raw_data_path + "features_file.pkl")
labels_file = pd.read_pickle(raw_data_path + "labels_file.pkl")
test_file = pd.read_pickle(raw_data_path + "test_file.pkl")

In [6]:
# select "transient" (supernovae) samples and save them as .pkl

transient_classes = ["SNIa", "SNIbc", "SNII", "SLSN"]
transient_labeled = labels_file[labels_file["classALeRCE"].isin(transient_classes)]
transient_features = features_file.loc[transient_labeled.index]
transient_labeled.to_pickle("data/transient_labeled.pkl")
transient_features.to_pickle("data/transient_features.pkl")

# select "stochastic" (supernovae) samples and save them as .pkl

stochastic_classes = ["QSO", "AGN", "Blazar", "YSO", "CV/Nova"]
stochastic_labeled = labels_file[labels_file["classALeRCE"].isin(stochastic_classes)]
stochastic_features = features_file.loc[stochastic_labeled.index]
stochastic_labeled.to_pickle("data/stochastic_labeled.pkl")
stochastic_features.to_pickle("data/stochastic_features.pkl")

# select "periodic" (variable stars) samples and save them as .pkl

periodic_classes = ["LPV","E","DSCT","RRL","CEP","Periodic-Other"]
periodic_labeled = labels_file[labels_file["classALeRCE"].isin(periodic_classes)]
periodic_features = features_file.loc[periodic_labeled.index]
periodic_labeled.to_pickle("data/periodic_labeled.pkl")
periodic_features.to_pickle("data/periodic_features.pkl")